In [1]:
import cv2
import face_recognition
from imutils import paths
import pickle
import numpy as np

In [2]:
image_paths =list(paths.list_images('Images'))
image_paths
known_names = []
known_encodings = []
# print(image_paths)

In [3]:
for image in image_paths:
    name = image.split('\\')[1]
    bgr_img = cv2.imread(image)
    #print(bgr_img)
    rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    
    faces = face_recognition.face_locations (rgb_img, model = 'hog')
    encodings = face_recognition.face_encodings(rgb_img, faces)
    
    for encoding in encodings:
        known_names.append(name)
        known_encodings.append(encoding)
        
data = {'names' : known_names, 'encodings' : known_encodings}

with open('face_encodings', 'wb') as f:
    f.write(pickle.dumps(data))

In [ ]:
cv2.imread()

In [ ]:
new_image = cv2.imread('group (2).jpg')
new_image = cv2.resize(new_image, None, fx = 0.25, fy = 0.25)
rgb_new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)

#classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

faces = face_recognition.face_locations(new_image,)

encodings = face_recognition.face_encodings(rgb_new_image, faces)

with open ('face_encodings', 'rb') as f:
    data = pickle.loads(f.read())

count_names = {}
names = []
for encoding in encodings:
    #matches = face_recognition.compare_faces(data['encodings'] , encoding)
    distances = face_recognition.face_distance(data['encodings'] , encoding)
    min_value = np.min(distances)
    if min_value > 0.3:
        name = 'Unknown'
    else:
        i = np.argmin(distances)
        name = data['names'][i]
    names.append(name)
    
    
   # if True in matches:
    #    match_idx = [i for i, match in enumerate(matches) if match]
     #  
      #  for i in match_idx:
       #     name = data['names'][i]
        #    count_names[name] = count_names.get(name, 0) + 1
        #name = max(count_names, key = count_names.get)
    #names.append(name)
    
print(names)
face_names = list(zip(faces, names))
for ((top,right,bottom,left), name) in face_names:
    cv2.rectangle(new_image, (left,top), (right, bottom), (255,0,0), 4)
    cv2.putText(new_image,name, (left-2, top-5), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (0,255,0), 2 )
    
cv2.imshow('Picture', new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()  
  
    


In [4]:
video = cv2.VideoCapture(0)
with open ('face_encodings', 'rb') as f:
    data = pickle.loads(f.read())
count_names = {}
names = []
while True:
    ret, frame = video.read()
    rgb_frame= cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_recognition.face_locations(frame)
    encodings = face_recognition.face_encodings(rgb_frame, faces)



    for encoding in encodings:
        matches = face_recognition.compare_faces(data['encodings'] , encoding)
        min_value = np.min(distances)
        if min_value > 0.3:
            name = 'Unknown'
        else:
            i = np.argmin(distances)
            name = data['names'][i]
        names.append(name)
            
    
        #if True in matches:
           # match_idx = [i for i, match in enumerate(matches) if match]
       
            #for i in match_idx:
                #name = data['names'][i]
               # count_names[name] = count_names.get(name, 0) + 1
            #name = max(count_names, key = count_names.get)
        #names.append(name)
    face_names = list(zip(faces, names))
    for ((top,right,bottom,left), name) in face_names:
        cv2.rectangle(frame, (left,top), (right, bottom), (255,0,0), 4)
        cv2.putText(frame,name, (left-2, bottom-5), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (0,255,0), 2 )
    cv2.imshow('face', frame)
    key = cv2.waitKey(100)
    if key ==ord('q'):
        break
video.release()
cv2.destroyAllWindows()